# Neo4j Notebook

Notebook criado para executar comandos do Neo4j fora do browser do banco.


In [3]:
from neo4j import GraphDatabase
import pandas as pd

In [4]:
class ConnectionNeo4j():

    #Abrindo conexao
    uri = "bolt://localhost:11003" #local BD
    password = "password" #local BD
    user = "neo4j"
    driver = GraphDatabase.driver(uri, auth=(user, password))

    #Fechando conexao
    def close(self):
        self.driver.close()

    #Salvando todos as possível para todas as cidades
    def salvarPathCidades(self, cityName):
        self.driver = GraphDatabase.driver(self.uri, auth=(self.user, self.password))

        with self.driver.session(database="neo4j") as session:
            result = session.read_transaction(self._pathCidade, cityName)

            return result
        

    #Funcao com a query de busca
    @staticmethod
    def _pathCidade(tx, cityName):
        query = (
            "WITH \"MATCH (cities:Cidade) "
            "WITH collect(cities) AS nodes"
            " UNWIND nodes AS ct"
            " MATCH (source:Cidade {cod_mun:"+cityName+"}), (target:Cidade {cod_mun: ct.cod_mun})"
            " CALL gds.shortestPath.dijkstra.stream('grafoFluxo', {"
            "     sourceNode: source,"
            "     targetNode: target,"
            "     relationshipWeightProperty: 'fluxo_geral'})"
            " YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path"
            " RETURN"
            "     index as index,"
            "     gds.util.asNode(sourceNode).cod_mun AS cod_mun_origem,"
            "     gds.util.asNode(targetNode).cod_mun AS cod_mun_destino,"
            "     totalCost as totalCost,"
            "     [nodeId IN nodeIds | gds.util.asNode(nodeId).cod_mun] AS cod_nos,"
            "     costs as costs\" AS query"
            f" CALL apoc.export.csv.query(query, \"cidades/{cityName}.csv\""+", {})"
            " YIELD file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data"
            " RETURN file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data;"
        )
        
        result = tx.run(query)
        return result.values("file")



In [5]:
dfCidades = pd.read_csv("dados_final/2_dados_sem_enriquecimento/arr_mun.csv")
listCodCidade = dfCidades.cod_cidade.to_list()

cnnNeo = ConnectionNeo4j()
for codigoCidade in listCodCidade:
    file = cnnNeo.salvarPathCidades(str(codigoCidade))
    print(file)
    
cnnNeo.close()

/var/folders/rv/x_8qm7392xv0jjj7dh82d9nm0000gn/T/ipykernel_33435/2667979410.py:18: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._pathCidade, cityName)


[['cidades/1100106.csv']]


/var/folders/rv/x_8qm7392xv0jjj7dh82d9nm0000gn/T/ipykernel_33435/2667979410.py:18: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._pathCidade, cityName)


[['cidades/1100205.csv']]


/var/folders/rv/x_8qm7392xv0jjj7dh82d9nm0000gn/T/ipykernel_33435/2667979410.py:18: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._pathCidade, cityName)


[['cidades/1200054.csv']]


/var/folders/rv/x_8qm7392xv0jjj7dh82d9nm0000gn/T/ipykernel_33435/2667979410.py:18: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(self._pathCidade, cityName)
